<a href="https://colab.research.google.com/github/narfdf999/Arquitectura/blob/master/producto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Solución final al problema de generar edificio a lo largo de un camino***




# Introducción

###"*Caminante, no hay camino. Se hace camino al andar*"
Antonio Machado *1912*


---

En los últimos meses se ha avanzado mucho en lo correspondiente a generación de plantas de edificios y su distribución. El enfoque el éste trabajo es radicalmente distinto. Nosotros pretendemos ponernos en la piel del transeunte y vivir la experiencia de recorrer los edificios y las sensaciones que te generan. Para ello se toma como input una curva paramétrica de 3 dimensiones que representa el camino, la dividimos en trozos muy pequeños y realizamos las visuales con forma de nube de puntos en cada una y que de una intuición al arquitecto de por donde seguir en su diseño. Una visual en arquitectura como su propio nombre indica es la percepción en 360º desde una posición determinada de un edificio. La forma más eficiente es, usando un sistema de coordenadas esféricas tomar ángulos fijos y calcular la tercera coordenada, que es el radio. De esta forma obtenemos una esfera de distancias que puede ser simplificada en una matriz de distancas usando una proyección de Mercator. Dichas matrices son lo que el modelo genera. Después de obtener todas las visuales correspondientes se calcula la transformación inversa a la proyección de Mercator y se transporta cada una al punto indicado en la curva. Así se genera la nube de puntos de todo el camino através del edificio como si de un hormiguero se tratara.


#Librerías

---



In [0]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import keras
from keras.layers import Dense, Dropout, Input
from keras.models import Model ,Sequential, load_model
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam

# Visualización y preprocesamiento de datos

---



# Implementación del sistema

---



### Optimizador ADAM

In [0]:
def adam_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

## DCGAN generadora de la primera visual

---

Para obtener una visual nueva por donde empezar y totalmente independiente de cualquier edificio se usa una GAN (Generative Adversarial Network) que será entrenada con miles de visuales de edificios de arquitectos famosos como Le Corbusier. Una GAN consta de dos redes, el generador y el discriminador. La primera se encarga de generar visuales a partir de un vector de ruido aleatorio. La segunda calcula si la visual ha sido generada mediante el generador o es una visual de un edificio real. Las dos redes van mejorando simultaneamente en sus tareas hasta que el generador es capaz de crear visuales que parecen reales. En ese momento se desecha el discriminador y se usa solo el generador para crear la primera pieza de dominó que servirá para ir creando el resto según se avance por el camino.



### Generador

El generador toma como input un vector de 128 posiciones de ruido aleatorio y lo transforma en un output con forma de matriz ( , ). Para ello se construye en base a una red deconvolucional. La matriz de salida representa una proyección de Mercator de la visual en 3d. 




In [0]:
# 

### Discriminador

### DCGAN

## Red neuronal que avanza paso a paso por el camino y genera las visuales según avanza

---



# Entrenamiento

---



#Modelo generado final

---

# Postprocesado de datos de salida

---

